# Objective Function

In [3]:
import numpy as np

def sphere (x) :
    return np.sum (np.square(x), axis=1)

def rastrigin (x) :
    A = 10
    return A*x.shape[1] + np.sum(np.square(x) - A*np.cos(2*np.pi*x) , axis=1)

# Chaos Definition

In [4]:
def logisticMap (x, r) :
    return r * x * (1 - x) 

def tentMap (x, mu) :
    if x <= mu :
        return x/mu
    else :
        return (1-x)/(1-mu)

def lorenzFlow (initCond, param, t_end, length) :
    from scipy.integrate import odeint
    def lorenz (X, t, sigma, beta, rho) :
        x, y, z = X
        dXdt = [sigma*(y - x), x*(rho - z) - y, x*y - beta*z]
        return dXdt

    X0 = initCond
    t = np.linspace (0, t_end, length)
    return odeint (lorenz, X0, t, args = param)

class chaosGenerator :    
    def setLorenz (self, sol) :
        self.x0 = sol[-1,0]
        self.y0 = sol[-1,1]
        self.z0 = sol[-1,2]
    
    def __init__ (self, cmap) :
        
        self.cmap = cmap
        
        if cmap[0] == "logistic" or cmap[0] == "tent" :
            self.x0 = 0.01
            iterate = 1000 + int(np.random.rand() * 10000)
            funcMap = logisticMap if cmap[0] == "logistic" else tentMap
            for i in range (1, iterate) :
                self.x0 = funcMap (self.x0, cmap[1])
                
        elif cmap[0] == "lorenz" :
            self.x0 = np.random.rand ()
            self.y0 = np.random.rand ()
            self.z0 = np.random.rand ()
            sol = lorenzFlow ([self.x0, self.y0, self.z0], cmap[1], 1000, 100000)
            self.xmin = min(sol[:,0])
            self.xmax = max(sol[:,0])
            self.setLorenz (sol)
                
    def getTimeSeries (self, length) :
        if self.cmap[0] == "logistic" or self.cmap[0] == "tent" :
            n = np.arange (1, length + 1)
            xs = [self.x0]
            x = self.x0
            
            funcMap = logisticMap if cmap[0] == "logistic" else tentMap
            for i in range (1, length) :
                x = funcMap (x, self.cmap[1])
                xs.append (x)

            xs = np.array (xs)
            self.x0 = xs[-1]
            xs = np.reshape (xs, (-1, 1))
            
        elif self.cmap[0] == "lorenz" :
            sol = lorenzFlow ([self.x0, self.y0, self.z0], cmap[1], length, length * 100)
            inds = np.arange (0, length*100, 100).astype (int)
            xs = np.reshape (sol[:,0][inds], (-1, 1))
            xs = (xs - self.xmin)/(self.xmax - self.xmin)
            
            self.setLorenz (sol)
            
        return xs

    def getChaosPoints (self, shape) :
        prod = 1
        for i in range(0, len(shape)) :
            prod *= shape[i]
            
        return self.getTimeSeries(prod).reshape(shape)
    
    def chaosRand (self) :
        if self.cmap[0] == "logistic" or self.cmap[0] == "tent" :
            funcMap = logisticMap if cmap[0] == "logistic" else tentMap
            self.x0 = funcMap (self.x0, cmap[1])
            
            return self.x0
        elif self.cmap[0] == "lorenz" :
            sol = lorenzFlow ([self.x0, self.y0, self.z0], cmap[1], 0.02, 2)
            self.setLorenz (sol)
            
            retx0 = (self.x0 - self.xmin)/(self.xmax - self.xmin)
            retx0 = 0 if retx0 < 0 else (1 if retx0 > 1 else retx0) 
            
            return retx0

## Update parameters

In [29]:
D = 5
Np = 20
w = 0.5
c1 = 2
c2 = 2

In [44]:
# cmap = ["lorenz", (10, 8.0/3, 28)]
cmap = ["logistic", 4]
chaosMan = chaosGenerator (cmap)

particles = 10.24 * chaosMan.getChaosPoints ((Np, D)) - 5.12
velocity = chaosMan.getChaosPoints ((Np, D))
pbest = particles
obj = rastrigin

gbest = min (particles , key = lambda x : obj(x.reshape(1,-1))[0])
print (gbest)

[ 3.92759506 -0.90578241  4.79951493 -3.87818107 -0.75511268]


In [45]:
for i in range (0, 1000) :
    r1 = chaosMan.getChaosPoints ((Np,D))
    r2 = chaosMan.getChaosPoints ((Np,D))
    velocity = w*velocity + c1*r1*(pbest - particles) + c2*r2*(gbest - particles)
    particles = particles + velocity
        
    less = obj(particles) < obj(pbest)
    pbest[less] = particles[less]
    
    gbest = min (pbest , key = lambda x : obj(x.reshape(1,-1))[0])
    
print ("f" + str(gbest) + " = " + str(obj(gbest.reshape(1,-1))[0]))

f[ 9.95127335e-01  2.00517418e-05 -1.48840297e-04 -3.95796301e-04
  8.77540959e-05] = 0.9950017819948371
